# NPS adjustments

Before trainning the model, there are a few caveats about the NPS and the touchpoints' data. Particularly, there are 3 adjustments we have to consider:
1) Monthly statistical adjustment (weights): Each month, the NPS scores per cabin/haul are adjusted to fit a true sample of the global distribution of cabin/haul users. That is, for the NPS per cabin haul to be representative, it is first adjusted so that the sample of users from which it is computed is representative as well. That way, if by chance a month has only one passenger in, for example, Bussines-Long Haul, the associated NPS considers that that particular sample is not representtive of the normal distribution of Business Long Haul users. Unfortunatelly, this values are computed every month, and we wont have access to them at prediction time. 
2) Change of survey: On early december 2019 and on late december 2023 there have been changes on the survey handled to the clients that have considerelly affected the NPS score and the satisfaction per touchpoint. A fixed value (per cabin/haul and globally) is used to adjust 2019 and 2022 (plus december 2019). Those values are computed externally.
3) C/H ponderation: On the global NPS computation, each cabin haul NPS is adjusted by a factor based on the percentaje of users that will fly on that particular cabine/haul. These values are fixed and at the moment of prediction we will have access to them.

## Installs and imports

In [2]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached boto3-1.19.12-py3-none-any.whl (131 kB)
  Using cached botocore-1.22.12-py3-none-any.whl (8.1 MB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.63
    Uninstalling botocore-1.31.63:
      Successfully uninstalled botocore-1.31.63
  Attempting uninstall: s3transfer
    Found exi

In [3]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")

## Read files

In [4]:
nps_path = 's3://iberia-data-lake/customer/nps_surveys/export_historic/insert_date_ci=2023-11-07/'
iberia_kpis_path = 's3://iberia-data-lake/customer/one_shot/iberia_kpis_yearly/'

In [5]:
import boto3
import pandas as pd

def read_csv_files_from_s3(s3_path):
    # Initialize Boto3 client for S3
    s3_client = boto3.client('s3')

    # Extract bucket name and prefix from the S3 path
    bucket, prefix = s3_path.replace('s3://', '').split('/', 1)

    # Get a list of all keys (object paths) under the specified S3 path
    s3_resource = boto3.resource('s3')
    s3_keys = [item.key for item in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)]

    # Generate a list of full S3 paths for all CSV files
    preprocess_paths = [f"s3://{bucket}/{key}" for key in s3_keys]

    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(s3_client.get_object(Bucket=bucket, Key=key)['Body']) for key in s3_keys]

    # Concatenate DataFrames into a single DataFrame
    df_concatenated = pd.concat(dfs, axis=0, ignore_index=True)

    return df_concatenated

In [6]:
# Readd files from paths
df_nps = read_csv_files_from_s3(nps_path)

In [7]:
df_nps[df_nps['monthly_weight'].notna()]['date_flight_local'].max()

'2023-09-30'

In [8]:
df_kpis = read_csv_files_from_s3(iberia_kpis_path)

In [9]:
df_kpis=df_kpis.groupby(['year', 'cabin', 'haul']).mean().reset_index()

In [10]:
df_kpis

,year,cabin,haul,load_factor,otp15_landing,otp15_takeoff
0,2019,Business,LH,82.353,79.387,86.456
1,2019,Business,SH,64.901,86.229,89.711
2,2019,Economy,LH,87.230,79.387,86.456
3,2019,Economy,SH,84.238,86.229,89.711
4,2019,Premium Economy,LH,80.994,79.387,86.456
5,2019,Premium Economy,SH,83.584,86.229,89.711
6,2020,Business,LH,66.909,87.109,88.509
7,2020,Business,SH,46.079,93.610,94.521
8,2020,Economy,LH,69.840,87.109,88.509
9,2020,Economy,SH,66.256,93.610,94.521


## 1) Monthly statistical adjustment.

In [11]:
condition_1 = (df_nps['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps['invitegroup_ib'] != 3) | (df_nps['invitegroup_ib'].isnull()))
condition_3 = (df_nps['invitegroup'] == 2)

df_nps_tkt = df_nps.loc[condition_1 & (condition_2 & condition_3)]

In [12]:
df_nps

,respondent_id,sample_id,surveyed_flight_number,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,tier_level,language_code,aircraft_registration_number,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,infinita_customers_identifer,flag_of_ib_singular_customers,country_code,list_of_options_for_booking_channel,list_of_options_for_checkin_channel,lounge_used_at_origin_airport,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,fleet_in_surveyed_flight,marketing_airline_code,date_of_flight_gmt,scheduled_departure_time_gmt,real_departure_time_local,real_departure_time_gmt,scheduled_arrival_time_gmt,real_arrival_time_local,real_arrival_time_gmt,segment,route,overall_haul,purser,invitegroup_ib,weight_category,weekly_weight,monthly_weight,pnr_show,ff_number,id_golden_record,ticket_num,started,time_spent_hrminsec,customer_email_show,origin_of_surveyed_flight,destination_of_surveyed_flight,operating_airline_code,cabin_in_surveyed_flight,haul,first_name_show,last_name_show,second_last_name_show,nps_category,nps_100,survey_type,invitegroup,group_age_survey,pun_100_punctuality,inm_200_issues_prior_checkin,inm_200_issues_prior_ticket_change,inm_200_issues_prior_schedule_change,inm_200_issues_prior_contact_center,inm_200_issues_prior_special_serv,inm_200_issues_prior_special_req,inm_200_issues_prior_avios,inm_200_issues_prior_voucher,inm_206_issues_checkin_long_queues,inm_206_issues_checkin_wrong_info,inm_206_issues_checkin_additional_fees,inm_206_issues_checkin_overbooking,inm_206_issues_checkin_downgrade,inm_206_issues_checkin_staff,inm_206_issues_checkin_social_distance,inm_206_issues_checkin_face_masks,inm_206_issues_checkin_documentation,inm_206_issues_checkin_other,inm_207_issues_lounge_denied,inm_207_issues_lounge_overcrowded,inm_207_issues_lounge_cleanliness,inm_207_issues_lounge_wifi,inm_207_issues_lounge_staff,inm_207_issues_lounge_food_drink,inm_207_issues_lounge_face_masks,inm_207_issues_lounge_other,inm_208_issues_security_leave_sth,inm_208_issues_security_long_queues,inm_208_issues_security_staff,inm_208_issues_security_social_distance,inm_208_issues_security_face_masks,inm_208_issues_security_other,inm_209_issues_boarding_unclear,inm_209_issues_boarding_gate_changed,inm_209_issues_boarding_lack_space,inm_209_issues_boarding_disorganised,inm_209_issues_boarding_staff,inm_209_issues_boarding_social_distance,inm_209_issues_boarding_face_masks,inm_209_issues_boarding_documentation,inm_209_issues_boarding_other,inm_220_issues_timing_cancelled,inm_220_issues_timing_dep_delay,inm_220_issues_timing_arr_delay,inm_220_issues_timing_missed,inm_230_issues_onboard_staff,inm_230_issues_onboard_ife,inm_230_issues_onboard_overcrowding,inm_230_issues_onboard_face_masks,inm_235_issues_onboard_comfort_damaged,inm_235_issues_onboard_comfort_space,inm_235_issues_onboard_comfort_temperature,inm_235_issues_onboard_comfort_cleanliness,inm_235_issues_onboard_comfort_washrooms,inm_235_issues_onboard_comfort_other,inm_236_issues_meal_availability,inm_236_issues_meal_portions,inm_236_issues_meal_quality,inm_236_issues_meal_special,inm_236_issues_meal_other,inm_240_issues_baggage_lost,inm_240_issues_baggage_delayed,inm_240_issues_baggage_demaged,inm_240_issues_baggage_staff,inm_240_issues_baggage_hand,inm_240_issues_baggage_other,inm_250_issues_arrival_slow,inm_250_issues_arrival_unclear,inm_250_issues_arrival_aditional_request,inm_250_issues_arrival_staff,inm_250_issues_arrival_immigration_queues,inm_250_issues_arrival_immigration_passport,inm_250_issues_arrival_immigration_other,inm_255_issues_connecting_missed,inm_255_issues_connecting_staff,inm_255_issues_connecting_baggage,inm_255_issues_connecting_other,bkg_100_booking,bkg_200_journey_preparation,inm_400_issues_response,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_100_cabin_crew,ifl_200_flight_crew_annoucements,ifl_300_cabin,

In [13]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender'] # invite_group

#'bkg_100_booking', 
touchpoints = ['bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

for feat in datetime_features:
    if feat in ['scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local', 'real_arrival_time_local','date_flight_local']:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], format="%Y%m%d %H:%M:%S", errors = 'coerce')
    else:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], errors = 'ignore')
df_nps_tkt['time_spent_hrminsec'] = pd.to_timedelta(df_nps_tkt['time_spent_hrminsec']).dt.total_seconds()
df_nps_tkt['started_hour'] = df_nps_tkt['started'].dt.hour
df_nps_tkt['year_flight'] = df_nps_tkt['date_flight_local'].dt.year
df_nps_tkt['month_flight'] = df_nps_tkt['date_flight_local'].dt.month
df_nps_tkt['day_flight'] = df_nps_tkt['date_flight_local'].dt.day
df_nps_tkt['weekday_flight'] = df_nps_tkt['date_flight_local'].dt.weekday
df_nps_tkt['is_weekend_or_friday_flight'] = df_nps_tkt['weekday_flight'].apply(lambda x: 1 if x in [5, 6,7] else 0)
df_nps_tkt['delay_departure'] = (df_nps_tkt['real_departure_time_local'] - df_nps_tkt['scheduled_departure_time_local']).dt.total_seconds()/60
df_nps_tkt['delay_arrival'] = (df_nps_tkt['real_arrival_time_local'] - df_nps_tkt['scheduled_arrival_time_local']).dt.total_seconds()/60
datetime_features = datetime_features + ['time_spent_hrminsec', 'started_hour', 'year_flight', 'month_flight',
                                         'day_flight', 'weekday_flight', 'is_weekend_or_friday_flight']

In [14]:
df_nps_tkt=df_nps_tkt[df_nps_tkt['date_flight_local'].dt.year >= 2019]

In [15]:
df_nps_tkt['date_flight_local'].min()

Timestamp('2019-01-01 00:00:00')

In [16]:
columns_to_select = datetime_features + columns_ext + touchpoints + ['delay_arrival', 'delay_departure', 'ticket_price']+columns_to_cross_kpis+['monthly_weight']

In [17]:
df_nps_tkt = df_nps_tkt[columns_to_select]
df_nps_tkt[['date_flight_local','nps_100','monthly_weight']]

,date_flight_local,nps_100,monthly_weight
1,2023-01-04,8,0.519
2,2023-01-29,10,0.414
3,2019-07-23,9,1.593
4,2019-09-03,6,0.729
6,2019-02-16,9,1.502
...,...,...,...
903274,2023-10-11,9,NaN
903275,2023-10-14,10,NaN
903276,2023-10-24,10,NaN
903277,2023-10-19,10,NaN


In [18]:
x=df_nps_tkt.copy()
pd.set_option('display.max_rows', 100)
x = x[(x['date_flight_local']=='2022-12-01') & (x['cabin_in_surveyed_flight'] == 'Economy') & (x['haul'] == 'MH') ]
display(x[(x['nps_100'] > 8) | (x['nps_100'] < 6)][['date_flight_local','cabin_in_surveyed_flight','haul','monthly_weight','nps_100']].head(200))



,date_flight_local,cabin_in_surveyed_flight,haul,monthly_weight,nps_100
14391,2022-12-01,Economy,MH,2.708,9
27782,2022-12-01,Economy,MH,2.708,10
52619,2022-12-01,Economy,MH,2.708,9
64431,2022-12-01,Economy,MH,2.708,9
64445,2022-12-01,Economy,MH,2.708,3
66099,2022-12-01,Economy,MH,2.708,5
66240,2022-12-01,Economy,MH,2.708,10
66271,2022-12-01,Economy,MH,2.708,5
87195,2022-12-01,Economy,MH,2.708,10
87393,2022-12-01,Economy,MH,2.708,10


#### If 4 cabin/haul

Before aggregating, I need to rewrite come categories in order to comapre with the dashboard:

In [19]:
df_nps_tkt['haul'] = df_nps_tkt['haul'].replace('MH', 'SH')
#df_nps_tkt['cabin_in_surveyed_flight'] = df_nps_tkt['cabin_in_surveyed_flight'].replace('Premium Economy', 'Economy')

#df_kpis['cabin']=df_kpis['cabin'].replace('Premium Economy','Premium Economy')
df_kpis['cabin']=df_kpis['cabin'].replace('Premium','Business')


In [20]:
df_nps_tkt[df_nps_tkt['monthly_weight'].isnull()][['date_flight_local','cabin_in_surveyed_flight','haul']]

,date_flight_local,cabin_in_surveyed_flight,haul
32769,2023-06-30,Economy,SH
33783,2023-07-31,Premium Economy,LH
33927,2023-07-31,Economy,SH
34417,2023-10-01,Economy,SH
34429,2023-10-02,Economy,SH
...,...,...,...
903274,2023-10-11,Business,SH
903275,2023-10-14,Economy,SH
903276,2023-10-24,Economy,SH
903277,2023-10-19,Economy,SH


In [21]:
df_nps_tkt = df_nps_tkt[df_nps_tkt['monthly_weight'].notnull()]

In [22]:
# df_nps_tkt = df_nps_tkt.query(
#     '(date_flight_local <= "2023-03-07") | '
#     '(date_flight_local >= "2023-03-14") & (date_flight_local <= "2023-03-23") | '
#     '(date_flight_local >= "2023-03-26")'
# )

### Aggregate on time frequency

In [23]:
# Function to calculate NPS
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features_with_cross_kpis(df, variables, columns_to_cross_kpis, time_frequency='Y', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Combine the values of columns_to_cross_kpis into a new column 'cross_kpis'
    df['cross_kpis'] = df[columns_to_cross_kpis].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)

    # Group the data by 'cross_kpis' and 'date_flight_local' using pd.Grouper on the entire DataFrame
    grouped_data = df.groupby([pd.Grouper(key='cross_kpis'), pd.Grouper(key='date_flight_local', freq=time_frequency)], group_keys=False)

    # Calculate the aggregated features for each group
    aggregated_features_data = pd.DataFrame()
    
    # Calculate the aggregated features for each group
    aggregated_features_data['NPS'] = grouped_data.apply(lambda x: calculate_nps(
        sum((x['nps_100'] == 9) | (x['nps_100'] == 10)),
        sum(x['nps_100'] <= 6),
        sum((x['nps_100'] == 7) | (x['nps_100'] == 8))
    ))

    # Calculate promoters_weight and detractors_weight
    aggregated_features_data['promoters_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] > 8]['monthly_weight'].sum()) if (x['nps_100'] > 8).any() else 0)
    aggregated_features_data['detractors_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] <= 6]['monthly_weight'].sum()) if (x['nps_100'] <= 6).any() else 0)




    # Calculate monthly_weight within each group
    aggregated_features_data['monthly_weight'] = grouped_data.apply(lambda x: x['monthly_weight'].sum())

    # Calculate NPS_weighted using the same logic as your SQL query
    aggregated_features_data['NPS_weighted'] = (aggregated_features_data['promoters_weight'] - aggregated_features_data['detractors_weight']) * 100 / aggregated_features_data['monthly_weight']

    for col in touchpoints + non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = grouped_data[col].sum()
        aggregated_features_data[f'{col}_std'] = grouped_data[col].std()
        aggregated_features_data[f'{col}_mean'] = grouped_data[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = grouped_data[col].apply(lambda x: x.notnull().sum())
        if col in touchpoints:
            aggregated_features_data[f'{col}_satisfaction'] = grouped_data[col].apply(lambda x: x[x >= 8].count() / x.count()*100)
        else:
            aggregated_features_data[f'{col}_max'] = grouped_data[col].max()
            aggregated_features_data[f'{col}_min'] = grouped_data[col].min()
    
    # Reset the index of aggregated_features_data
    aggregated_features_data.reset_index(inplace=True)

    # Split the 'cross_kpis' column back into individual columns and concatenate them to aggregated_features_data
    split_columns = aggregated_features_data['cross_kpis'].str.split('_', expand=True)
    split_columns.columns = columns_to_cross_kpis
    aggregated_features_data = pd.concat([split_columns, aggregated_features_data], axis=1)
    
    # Drop the temporary 'cross_kpis' column
    aggregated_features_data.drop(columns=['cross_kpis'], inplace=True)
    
    return aggregated_features_data


In [24]:
df=calculate_aggregated_features_with_cross_kpis(df_nps_tkt, touchpoints, columns_to_cross_kpis, time_frequency='Y', calculate_satisfaction=True)

In [25]:
df

,cabin_in_surveyed_flight,haul,date_flight_local,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction
0,Business,LH,2019-12-31,47.216,1732.415,421.364,2801.804,46.793,35804.000,2.252,8.255,4337,76.943,44217.000,1.924,8.541,5177,82.268,42650.000,1.896,8.391,5083,78.536,33825.000,2.162,7.912,4275,68.865,42422.000,2.107,8.110,5231,73.103,43953.000,1.838,8.393,5237,78.766,43935.000,1.738,8.523,5155,80.272,9794.000,3.345,5.370,1824,33.991,24726.000,1.864,7.933,3117,68.912,40696.000,2.157,7.858,5179,68.411,44601.000,1.979,8.515,5238,79.973,41783.000,1.947,8.156,5123,74.449,3052.000,2.267,7.846,389,71.979,46518.000,2.062,8.871,5244,86.575,33994.000,2.061,7.862,4324,67.299,2088.000,3.342,4.031,518,19.498,29421.000,2.544,7.072,4160,53.990
1,Business,LH,2020-12-31,25.896,459.604,177.444,845.394,33.376,13129.000,2.469,7.943,1653,71.627,15052.000,2.216,8.207,1834,76.554,14781.000,2.099,8.135,1817,74.408,8507.000,2.271,7.812,1089,66.850,14512.000,2.308,7.819,1856,69.343,15034.000,2.161,8.065,1864,73.766,15190.000,2.046,8.233,1845,75.610,3681.000,3.447,4.695,784,28.316,11206.000,2.109,7.665,1462,64.159,11580.000,3.192,6.321,1832,47.434,14722.000,2.470,7.911,1861,70.553,14170.000,2.235,7.773,1823,67.965,3535.000,2.599,7.334,482,62.863,16784.000,1.877,8.980,1869,88.175,8027.000,2.043,7.778,1032,65.988,847.000,3.388,3.798,223,18.834,10158.000,2.884,6.549,1551,47.002
2,Business,LH,2021-12-31,25.247,978.107,486.026,1942.596,25.331,21241.000,2.804,7.498,2833,65.161,24651.000,2.410,7.970,3093,72.583,25107.000,2.003,8.208,3059,75.155,14505.000,2.565,7.477,1940,61.598,24460.000,2.329,7.862,3111,69.592,25122.000,2.230,8.042,3124,73.079,25438.000,2.039,8.305,3063,76.330,7396.000,3.379,

### Merge dfs

In [26]:
# Convert the 'date_flight_local' column in result_df to separate 'year'column
df['year'] = df['date_flight_local'].dt.year

# Perform the merge based on the specified columns
merged_df = df.merge(df_kpis, 
                            left_on=['year', 'cabin_in_surveyed_flight', 'haul'], 
                            right_on=['year', 'cabin', 'haul'], 
                            how='inner')

# Drop the redundant columns from the merged dataframe
merged_df.drop(columns=['cabin_in_surveyed_flight'], inplace=True)

# Reorder the columns to place 'year', 'month', 'cabin', and 'haul' at the beginning
cols_to_move = ['date_flight_local','year', 'cabin', 'haul']
merged_df = merged_df[cols_to_move + [col for col in merged_df.columns if col not in cols_to_move]]
merged_df.head()

,date_flight_local,year,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-12-31,2019,Business,LH,47.216,1732.415,421.364,2801.804,46.793,35804.000,2.252,8.255,4337,76.943,44217.000,1.924,8.541,5177,82.268,42650.000,1.896,8.391,5083,78.536,33825.000,2.162,7.912,4275,68.865,42422.000,2.107,8.110,5231,73.103,43953.000,1.838,8.393,5237,78.766,43935.000,1.738,8.523,5155,80.272,9794.000,3.345,5.370,1824,33.991,24726.000,1.864,7.933,3117,68.912,40696.000,2.157,7.858,5179,68.411,44601.000,1.979,8.515,5238,79.973,41783.000,1.947,8.156,5123,74.449,3052.000,2.267,7.846,389,71.979,46518.000,2.062,8.871,5244,86.575,33994.000,2.061,7.862,4324,67.299,2088.000,3.342,4.031,518,19.498,29421.000,2.544,7.072,4160,53.990,82.353,79.387,86.456
1,2020-12-31,2020,Business,LH,25.896,459.604,177.444,845.394,33.376,13129.000,2.469,7.943,1653,71.627,15052.000,2.216,8.207,1834,76.554,14781.000,2.099,8.135,1817,74.408,8507.000,2.271,7.812,1089,66.850,14512.000,2.308,7.819,1856,69.343,15034.000,2.161,8.065,1864,73.766,15190.000,2.046,8.233,1845,75.610,3681.000,3.447,4.695,784,28.316,11206.000,2.109,7.665,1462,64.159,11580.000,3.192,6.321,1832,47.434,14722.000,2.470,7.911,1861,70.553,14170.000,2.235,7.773,1823,67.965,3535.000,2.599,7.334,482,62.863,16784.000,1.877,8.980,1869,88.175,8027.000,2.043,7.778,1032,65.988,847.000,3.388,3.798,223,18.834,10158.000,2.884,6.549,1551,47.002,66.909,87.109,88.509
2,2021-12-31,2021,Business,LH,25.247,978.107,486.026,1942.596,25.331,21241.000,2.804,7.498,2833,65.161,24651.000,2.410,7.970,3093,72.583,25107.000,2.003,8.208,3059,75.155,14505.000,2.565,7.477,1940,61.598,24460.000,2.329,7.862,3111,69.592,

In [27]:
merged_df.describe()

,year,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000
mean,2021.077,29.670,10564.283,4581.697,19814.468,30.010,135904.692,2.718,7.484,18136.269,65.016,162451.462,2.422,7.958,20458.423,71.942,167891.231,2.068,8.240,20342.692,76.247,18420.654,2.298,7.915,2323.423,70.278,162680.538,2.344,7.867,20683.577,70.663,164301.654,2.201,7.969,20738.308,70.402,168131.308,2.042,8.273,20292.538,75.242,21243.885,3.451,4.947,3931.885,30.183,43239.769,2.612,7.363,5587.231,63.177,92363.346,2.772,6.894,13727.885,52.551,168982.038,2.294,8.191,20405.577,75.337,162989.577,2.260,7.982,20408.038,71.374,44529.923,2.795,7.494,6064.577,65.593,172848.731,2.556,8.332,20982.577,80.108,69417.308,2.275,7.464,9471.577,59.602,8000.769,3.239,3.197,2730.385,13.979,60860.846,3.173,6.236,10059.038,46.438,73.364,85.716,87.791
std,1.468,15.599,15835.349,6778.690,29608.285,15.710,197046.401,0.223,0.546,26161.290,8.246,239301.072,0.236,0.386,29902.777,9.138,246373.525,0.147,0.211,29780.376,3.773,14337.510,0.1

In [28]:
merged_df['date_flight_local'].max()

Timestamp('2023-12-31 00:00:00')

In [29]:
merged_df.to_csv('outputs/yearly_NPS_5_cabin_haul_complete', index=False)

## 2) Change of survey adjustment.

Since the adjustment is done separatelly in the global NPS and in the Cabin/haul NPS we need to have a separate dataframe for the NPS.  

In [30]:
# Sample adjustments DataFrame
adjustments_data = {'haul': ['LH', 'LH', 'LH', 'SH', 'SH', 'SH'],
                    'cabin': ['Economy', 'Premium Economy' , 'Business', 'Economy', 'Premium Economy', 'Business'],
                    'adjustment_2019': [-0.55475719, -0.55475719, -6.86337594, -5.25927601, -5.25927601, -5.64891133],
                    'adjustment_2022': [1.41878561, 1.41878561, 0.42510779, -3.71320153, -3.71320153, -4.10337349]}

adjustments_df = pd.DataFrame(adjustments_data)
df=merged_df.copy()

for index, row in adjustments_df.iterrows():
    condition = ((df['haul'] == row['haul']) | (df['haul'] == 'MH')) & (df['cabin'] == row['cabin'])
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month != 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2019']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2019']
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month == 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
    mask = (df['date_flight_local'].dt.year == 2022)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
df[df['date_flight_local'] == '2022-12-31'].head()


,date_flight_local,year,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
3,2022-12-31,2022,Business,LH,37.124,1585.166,541.118,2773.208,38.073,19884.000,2.715,7.506,2649,65.383,24114.000,2.310,8.100,2977,74.840,23908.000,2.109,8.146,2935,74.549,18419.000,2.350,7.801,2361,67.556,24485.000,2.062,8.151,3004,74.501,24832.000,1.982,8.255,3008,76.330,25275.000,1.860,8.542,2959,80.500,8182.000,3.406,5.327,1536,34.375,17703.000,2.176,7.667,2309,64.747,22339.000,2.461,7.511,2974,62.307,25397.000,2.107,8.446,3007,78.650,23650.000,2.116,8.020,2949,73.008,5890.000,2.812,7.165,822,59.611,26272.000,2.265,8.725,3011,84.258,16591.000,1.994,7.699,2155,63.434,1613.000,3.521,3.840,420,19.762,15475.000,3.078,6.146,2518,42.891,87.386,82.678,83.950
8,2022-12-31,2022,Business,SH,33.539,4007.132,1435.398,6840.906,33.490,41791.000,2.695,7.535,5546,65.200,52063.000,2.316,8.084,6440,74.907,51610.000,2.185,8.122,6354,74.819,32515.000,2.241,7.985,4072,71.464,51508.000,2.372,7.785,6616,68.954,53218.000,2.125,8.040,6619,73.213,53273.000,2.067,8.295,6422,75.958,6946.000,3.366,5.469,1270,34.882,2208.000,3.417,6.255,353,46.742,47079.000,2.353,7.713,6104,65.858,57439.000,1.867,8.696,6605,83.982,51402.000,2.219,7.915,6494,70.342,17365.000,2.808,7.434,2336,64.683,55286.000,2.667,8.291,6668,78.884,31123.000,2.102,7.490,4155,57.810,2622.000,3.279,3.148,833,15.126,26911.000,3.189,5.913,4551,39.596,68.336,88.362,89.430
13,2022-12-31,2022,Economy,LH,13.005,12043.200,9000.470,27504.387,12.481,122458.000,3.071,6.913,17715,56.094,137769.000,2.844,7.316,18831,61.909,150006.000,2.262,8.044,18649,72.792,12635.000,2.397,7.747,1631,67.443,144314.000,2.405,

Lets check the NPS curve now against the dashboard:

In [31]:
shp_bi = pd.read_excel('inputs/SH Premium.xlsx')
shp_bi

,Quarter,Month_Name,2019,2022,2023
0,Q1,Jan,38.240,44.246,35.708
1,Q1,Feb,46.748,34.647,48.818
2,Q1,Mar,40.184,38.806,35.570
3,Q2,Apr,36.272,32.042,45.796
4,Q2,May,46.937,27.681,44.219
5,Q2,Jun,18.644,25.831,40.074
6,Q3,Jul,33.755,28.475,41.470
7,Q3,Aug,35.874,36.682,48.652
8,Q3,Sep,33.372,28.024,NaN
9,Q4,Oct,43.783,44.776,NaN


In [32]:
data=df[(df['date_flight_local'].dt.year.isin([2019, 2022, 2023])) & (df['cabin'] == 'Business') & (df['haul'] == 'SH')][['NPS', 'NPS_weighted', 'date_flight_local']]


In [33]:
# Pivot the dataframe
pivot_df = data.pivot_table(index=data['date_flight_local'].dt.month, columns=data['date_flight_local'].dt.year, values=['NPS', 'NPS_weighted'], aggfunc='mean')

# Rename the columns
pivot_df.columns = [f"{year}" for _, year in pivot_df.columns]

# Reset the index
pivot_df.reset_index(inplace=True)


pivot_df.rename(columns={'date_flight_local': 'Month_Name'}, inplace=True)

In [34]:
pivot_df

,Month_Name,2019,2022,2023,2019,2022,2023
0,12,37.530,33.539,42.255,38.185,33.490,42.148


In [35]:
df.to_csv('outputs/yearly_NPS_5ch_adjusted_complete', index=False)

#### Touchpoints' satisfaction  adjustment for the change of survey

In [36]:
df=pd.read_csv('outputs/yearly_NPS_5ch_adjusted_complete')
df.head()

,date_flight_local,year,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-12-31,2019,Business,LH,47.641,1732.415,421.364,2801.804,47.218,35804.000,2.252,8.255,4337,76.943,44217.000,1.924,8.541,5177,82.268,42650.000,1.896,8.391,5083,78.536,33825.000,2.162,7.912,4275,68.865,42422.000,2.107,8.110,5231,73.103,43953.000,1.838,8.393,5237,78.766,43935.000,1.738,8.523,5155,80.272,9794.000,3.345,5.370,1824,33.991,24726.000,1.864,7.933,3117,68.912,40696.000,2.157,7.858,5179,68.411,44601.000,1.979,8.515,5238,79.973,41783.000,1.947,8.156,5123,74.449,3052.000,2.267,7.846,389,71.979,46518.000,2.062,8.871,5244,86.575,33994.000,2.061,7.862,4324,67.299,2088.000,3.342,4.031,518,19.498,29421.000,2.544,7.072,4160,53.990,82.353,79.387,86.456
1,2020-12-31,2020,Business,LH,25.896,459.604,177.444,845.394,33.376,13129.000,2.469,7.943,1653,71.627,15052.000,2.216,8.207,1834,76.554,14781.000,2.099,8.135,1817,74.408,8507.000,2.271,7.812,1089,66.850,14512.000,2.308,7.819,1856,69.343,15034.000,2.161,8.065,1864,73.766,15190.000,2.046,8.233,1845,75.610,3681.000,3.447,4.695,784,28.316,11206.000,2.109,7.665,1462,64.159,11580.000,3.192,6.321,1832,47.434,14722.000,2.470,7.911,1861,70.553,14170.000,2.235,7.773,1823,67.965,3535.000,2.599,7.334,482,62.863,16784.000,1.877,8.980,1869,88.175,8027.000,2.043,7.778,1032,65.988,847.000,3.388,3.798,223,18.834,10158.000,2.884,6.549,1551,47.002,66.909,87.109,88.509
2,2021-12-31,2021,Business,LH,25.247,978.107,486.026,1942.596,25.331,21241.000,2.804,7.498,2833,65.161,24651.000,2.410,7.970,3093,72.583,25107.000,2.003,8.208,3059,75.155,14505.000,2.565,7.477,1940,61.598,24460.000,2.329,7.862,3111,69.592,

In [37]:
df['date_flight_local']=pd.to_datetime(df['date_flight_local'])

In [38]:
import pandas as pd
from io import StringIO

#Flight Booking
data = """
Touchpoint
IB Plus loyalty program
Boarding
Ease of contact by phone
IFE
Connections experience
In flight food and beverage
Airport security
Wi-Fi
Lounge
Arrivals experience
Check-in
Journey preparation support
Pilot's announcements
Aircraft interior
Cabin Crew
Punctuality
Response provided to the issue
"""
#5.0
data2 = """
Global adjustment
12.7
8.2
7.0
6.9
5.7
5.6
5.2
5.2
5.1
4.5
3.3
3.2
2.5
2.4
1.1
-0.2
-3.1
"""
#3.6
data3 = """
SH_Business
16.1
4.0
0.8
1.0
3.3
4.0
1.5
10.2
1.5
3.1
-1.2
4.8
-0.7
0.5
-3.0
-2.5
-2.6
"""
#7.5
data4 = """
LH_Business
11.3
5.5
11.4
5.0
12.6
3.3
14.0
2.3
8.2
6.2
0.8
7.9
1.2
2.1
3.6
1.2
3.4
"""
#4.6
data5 = """
SH_Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#4.6
data5 = """
SH_Premium Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#7.4
data6 = """
LH_Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""
#7.4
data7 = """
LH_Premium Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""

# Create StringIO objects to simulate file-like input
data_stream = StringIO(data)
data2_stream = StringIO(data2)
data3_stream = StringIO(data3)
data4_stream = StringIO(data4)
data5_stream = StringIO(data5)
data6_stream = StringIO(data6)
data7_stream = StringIO(data7)

# Create DataFrames
ta_df = pd.DataFrame({'Touchpoint': data_stream.getvalue().split('\n')[1:]})
ta_df['Global adjustment'] = data2_stream.getvalue().split('\n')[1:]
ta_df['SH_Business'] = data3_stream.getvalue().split('\n')[1:]
ta_df['LH_Business'] = data4_stream.getvalue().split('\n')[1:]
ta_df['SH_Economy'] = data5_stream.getvalue().split('\n')[1:]
ta_df['LH_Economy'] = data6_stream.getvalue().split('\n')[1:]
ta_df['LH_Premium Economy'] = data7_stream.getvalue().split('\n')[1:]

# Display the DataFrame
ta_df=ta_df.drop(0)
ta_df=ta_df.drop(18)
ta_df


,Touchpoint,Global adjustment,SH_Business,LH_Business,SH_Economy,LH_Economy,LH_Premium Economy
1,IB Plus loyalty program,12.7,16.1,11.3,12.7,11.2,11.2
2,Boarding,8.2,4.0,5.5,8.1,10.5,10.5
3,Ease of contact by phone,7.0,0.8,11.4,5.2,10.1,10.1
4,IFE,6.9,1.0,5.0,3.4,7.4,7.4
5,Connections experience,5.7,3.3,12.6,4.2,10.0,10.0
6,In flight food and beverage,5.6,4.0,3.3,5.6,8.6,8.6
7,Airport security,5.2,1.5,14.0,3.8,9.6,9.6
8,Wi-Fi,5.2,10.2,2.3,6.5,1.8,1.8
9,Lounge,5.1,1.5,8.2,6.7,9.8,9.8
10,Arrivals experience,4.5,3.1,6.2,3.8,7.5,7.5


In [39]:
satisfaction_mapping = {
    'IB Plus loyalty program': 'loy_200_loyalty_programme_satisfaction',
    'Boarding': 'pfl_500_boarding_satisfaction',
    'Ease of contact by phone': 'img_310_ease_contact_phone_satisfaction',
    'IFE': 'ifl_500_ife_satisfaction',
    'Connections experience': 'con_100_connections_satisfaction',
    'In flight food and beverage': 'ifl_400_food_drink_satisfaction',
    'Airport security': 'pfl_200_security_satisfaction',
    'Wi-Fi': 'ifl_600_wifi_satisfaction',
    'Lounge': 'pfl_300_lounge_satisfaction',
    #'Flight Booking': 'bkg_100_booking_satisfaction',
    'Arrivals experience': 'arr_100_arrivals_satisfaction',
    'Check-in': 'pfl_100_checkin_satisfaction',
    'Journey preparation support': 'bkg_200_journey_preparation_satisfaction',
    "Pilot's announcements": 'ifl_200_flight_crew_annoucements_satisfaction',
    'Aircraft interior': 'ifl_300_cabin_satisfaction',
    'Cabin Crew': 'ifl_100_cabin_crew_satisfaction',
    'Punctuality': 'pun_100_punctuality_satisfaction',
    'Response provided to the issue': 'inm_400_issues_response_satisfaction'
}

# Create a Series to use for mapping
satisfaction_series = pd.Series(satisfaction_mapping)

# Replace touchpoints with their satisfaction equivalents in the original DataFrame
ta_df['Touchpoint'] = ta_df['Touchpoint'].map(satisfaction_series)


#Flip the dataframe to match the NPS one:
flipped_df = ta_df.transpose()
flipped_df.columns = flipped_df.iloc[0]
ta_df = flipped_df.drop(flipped_df.index[0])
ta_df


Touchpoint,loy_200_loyalty_programme_satisfaction,pfl_500_boarding_satisfaction,img_310_ease_contact_phone_satisfaction,ifl_500_ife_satisfaction,con_100_connections_satisfaction,ifl_400_food_drink_satisfaction,pfl_200_security_satisfaction,ifl_600_wifi_satisfaction,pfl_300_lounge_satisfaction,arr_100_arrivals_satisfaction,pfl_100_checkin_satisfaction,bkg_200_journey_preparation_satisfaction,ifl_200_flight_crew_annoucements_satisfaction,ifl_300_cabin_satisfaction,ifl_100_cabin_crew_satisfaction,pun_100_punctuality_satisfaction,inm_400_issues_response_satisfaction
Global adjustment,12.7,8.2,7.0,6.9,5.7,5.6,5.2,5.2,5.1,4.5,3.3,3.2,2.5,2.4,1.1,-0.2,-3.1
SH_Business,16.1,4.0,0.8,1.0,3.3,4.0,1.5,10.2,1.5,3.1,-1.2,4.8,-0.7,0.5,-3.0,-2.5,-2.6
LH_Business,11.3,5.5,11.4,5.0,12.6,3.3,14.0,2.3,8.2,6.2,0.8,7.9,1.2,2.1,3.6,1.2,3.4
SH_Economy,12.7,8.1,5.2,3.4,4.2,5.6,3.8,6.5,6.7,3.8,3.2,3.0,2.3,2.7,1.5,-0.8,-3.8
LH_Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1
LH_Premium Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1


In [40]:
df.describe()

,year,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000
mean,2021.077,29.320,10564.283,4581.697,19814.468,29.659,135904.692,2.718,7.484,18136.269,65.016,162451.462,2.422,7.958,20458.423,71.942,167891.231,2.068,8.240,20342.692,76.247,18420.654,2.298,7.915,2323.423,70.278,162680.538,2.344,7.867,20683.577,70.663,164301.654,2.201,7.969,20738.308,70.402,168131.308,2.042,8.273,20292.538,75.242,21243.885,3.451,4.947,3931.885,30.183,43239.769,2.612,7.363,5587.231,63.177,92363.346,2.772,6.894,13727.885,52.551,168982.038,2.294,8.191,20405.577,75.337,162989.577,2.260,7.982,20408.038,71.374,44529.923,2.795,7.494,6064.577,65.593,172848.731,2.556,8.332,20982.577,80.108,69417.308,2.275,7.464,9471.577,59.602,8000.769,3.239,3.197,2730.385,13.979,60860.846,3.173,6.236,10059.038,46.438,73.364,85.716,87.791
std,1.468,15.425,15835.349,6778.690,29608.285,15.540,197046.401,0.223,0.546,26161.290,8.246,239301.072,0.236,0.386,29902.777,9.138,246373.525,0.147,0.211,29780.376,3.773,14337.510,0.1

In [41]:
original_df=df.copy()
adjustment_df=ta_df.drop("Global adjustment")

# Convert the values in the adjustment_df to numeric
adjustment_df = adjustment_df.apply(pd.to_numeric, errors='coerce')

# Filter out the rows where the year is 2023
adjusted_years = original_df['date_flight_local'].dt.year != 2023
adjusted_df = original_df[adjusted_years].copy()

# Add the adjustments to the corresponding touchpoint columns
for index, row in adjustment_df.iterrows():
    haul, cabin = index.split('_')
    mask = (adjusted_df['haul'] == haul) & (adjusted_df['cabin'] == cabin)
    for touchpoint_col in adjustment_df.columns:
        adjusted_df[touchpoint_col].fillna(0, inplace=True) 
        adjusted_df.loc[mask, touchpoint_col] += row[touchpoint_col]
        
# Concatenate adjusted_df with values from 2023
adjusted_df = pd.concat([adjusted_df, original_df[~adjusted_years]])




In [42]:
adjusted_df.describe()

,year,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,25.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000,25.000,26.000,26.000,26.000,26.000,26.000,26.000
mean,2021.077,29.320,10564.283,4581.697,19814.468,29.659,135904.692,2.718,7.484,18136.269,68.539,162451.462,2.422,7.958,20458.423,74.527,167891.231,2.068,8.240,20342.692,82.170,18420.654,2.298,7.915,2323.423,76.038,162680.538,2.344,7.867,20683.577,76.602,164301.654,2.201,7.969,20738.308,72.571,168131.308,2.042,8.273,20292.538,76.749,21243.885,3.451,4.947,3931.885,33.660,43239.769,2.612,7.363,5587.231,66.900,92363.346,2.772,6.894,13727.885,57.182,168982.038,2.294,8.191,20405.577,76.860,162989.577,2.260,7.982,20408.038,75.697,44529.923,2.795,7.494,6064.577,71.762,172848.731,2.556,8.332,20982.577,80.308,69417.308,2.275,7.464,9471.577,69.602,8000.769,3.239,3.197,2730.385,13.179,60860.846,3.173,6.236,10059.038,52.223,73.364,85.716,87.791
std,1.468,15.425,15835.349,6778.690,29608.285,15.540,197046.401,0.223,0.546,26161.290,9.179,239301.072,0.236,0.386,29902.777,9.132,246373.525,0.147,0.211,29780.376,5.774,14337.510,0.1

In [43]:
adjusted_df.to_csv('outputs/yearly_NPS_5ch_Touchpoints_adjusted_complete', index=False)